# Pool data exog and endo Distributions

How does the data look?

In [6]:
import os
os.chdir("/home/charles/src/LondonMirror/Prepayments/")

import prepayments as pp
import numpy as np
import pandas as pd
import ggplot as gg
from datetime import datetime

In [2]:
pm = pp.models.PoolModel()

In [3]:
samples = [(pm.data_pool.h5file.root.pools[i]['pool_number'],
            datetime.fromtimestamp(pm.data_pool.h5file.root.pools[i]['as_of_date']))
           for i in np.random.choice(14151801,1000)]

In [4]:
%timeit data = {(i,dt):pm.data_for_pool_number(i,dt) for i,dt in samples}

ERROR:root:Weighted Average Coupon is missing for 793381, 2014-06-01
ERROR:root:Weighted Average Coupon is missing for 649882, 2012-09-01
ERROR:root:Weighted Average Coupon is missing for 634726, 2012-06-01
ERROR:root:Weighted Average Coupon is missing for 618939, 2013-07-01
ERROR:root:Weighted Average Coupon is missing for 686536, 2013-11-01
ERROR:root:Weighted Average Coupon is missing for 669518, 2016-11-01
ERROR:root:Various table has no records for 783743, 2015-06-01
ERROR:root:Various table has no records for 782476, 2014-07-01
ERROR:root:Various table has no records for 781580, 2014-05-01
ERROR:root:Various table has no records for 780609, 2015-06-01
ERROR:root:Weighted Average Coupon is missing for 649758, 2012-08-01
ERROR:root:Weighted Average Coupon is missing for 797615, 2015-12-01
ERROR:root:Weighted Average Coupon is missing for 749177, 2012-03-01
ERROR:root:Weighted Average Coupon is missing for AB8081, 2013-03-01
ERROR:root:Weighted Average Coupon is missing for AG5017, 

1 loop, best of 3: 12min 3s per loop


Whoops, didn't realize timeit defaulted to multiple loops.  It actually ran 4 times. So there are really 58 missing records, or 5.8% of the total. Still pretty high.

12 minutes per thousand is also slower than I'd like, but probably manageable. I expect I can optimize more if necessary.

In [4]:
data = {(i,dt):pm.data_for_pool_number(i,dt) for i,dt in samples}

TypeError: argument of type 'NoneType' is not iterable

In [5]:
%debug

> /home/charles/src/LondonMirror/Prepayments/prepayments/bloomberg_data.py(245)get_cpr_for_cusip()
    243         #Check if we already have it
    244         toc = self.hdfstore.get_node(self.cpr_root)
--> 245         if cusip in toc:
    246             data = self.hdfstore[self.cpr_root + '_' + cusip]
    247         else:

ipdb> self.hdfstore
<class 'pandas.io.pytables.HDFStore'>
File path: data/cpr_data.h5
Empty
ipdb> q


In [6]:
pm.data_cpr.get_all_data()

In [7]:
pm.data_cpr.hdfstore.get_node(pm.data_cpr.cpr_root)

In [10]:
missing = [k for k,v in data.items() if v is None]
data = pd.DataFrame.from_records({k:v for k,v in data.items() if v is not None})
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9 entries, burnout to upfront_mip
Columns: 942 entries, (002508, 2013-09-01 00:00:00) to (MA3453, 2016-11-01 00:00:00)
dtypes: object(942)
memory usage: 66.3+ KB


In [11]:
data = data.T
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 942 entries, (002508, 2013-09-01 00:00:00) to (MA3453, 2016-11-01 00:00:00)
Data columns (total 9 columns):
burnout        942 non-null object
cato           854 non-null object
endo           942 non-null object
hpa            942 non-null object
incentive      942 non-null object
lockin         942 non-null object
sato           942 non-null object
seasonality    942 non-null object
upfront_mip    942 non-null object
dtypes: object(9)
memory usage: 73.6+ KB


In [13]:
data.index = pd.MultiIndex.from_tuples(data.index)
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 942 entries, (002508, 2013-09-01 00:00:00) to (MA3453, 2016-11-01 00:00:00)
Data columns (total 9 columns):
burnout        942 non-null object
cato           854 non-null object
endo           942 non-null object
hpa            942 non-null object
incentive      942 non-null object
lockin         942 non-null object
sato           942 non-null object
seasonality    942 non-null object
upfront_mip    942 non-null object
dtypes: object(9)
memory usage: 76.8+ KB


In [14]:
data.head()

,,burnout,cato,endo,hpa,incentive,lockin,sato,seasonality,upfront_mip
002508,2013-09-01,[1392478.09543],2.0742,[0.0],-0.0286831,[7292.628269],[0.00046213418274],[3194.01466422],9,15.2495
002808,2014-12-01,[1348789.60506],1.2257,[0.000328841090218],7.60616e+06,[7292.89830949],[0.000425122055222],[3170.54175189],12,13.7213
003576,2014-01-01,[618141.219292],0.4129,[5.62017744336e-05],2.86903e+06,[5376.56558002],[0.000638368025709],[1147.67365718],1,9.24308
003589,2014-11-01,[670407.931404],-0.2079,[0.0],7.20413e+06,[5364.80981254],[0.000594297216298],[1180.58740092],11,10.9246
003937,2014-11-01,[634081.247601],1.804,[0.0],-0.250354,[6746.76233637],[0.000479653871844],[2562.58740092],11,15.3846


In [16]:
data.endo = [x[0] for x in data.endo]
data.burnout = [x[0] for x in data.burnout]
data.incentive = [x[0] for x in data.incentive]
data.lockin = [x[0] for x in data.lockin]
data.sato = [x[0] for x in data.sato]

TypeError: 'int' object has no attribute '__getitem__'

In [18]:
data

,,burnout,cato,endo,hpa,incentive,lockin,sato,seasonality,upfront_mip
002508,2013-09-01,[1392478.09543],2.0742,0.000000,-0.0286831,[7292.628269],[0.00046213418274],[3194.01466422],9,15.2495
002808,2014-12-01,[1348789.60506],1.2257,0.000329,7.60616e+06,[7292.89830949],[0.000425122055222],[3170.54175189],12,13.7213
003576,2014-01-01,[618141.219292],0.4129,0.000056,2.86903e+06,[5376.56558002],[0.000638368025709],[1147.67365718],1,9.24308
003589,2014-11-01,[670407.931404],-0.2079,0.000000,7.20413e+06,[5364.80981254],[0.000594297216298],[1180.58740092],11,10.9246
003937,2014-11-01,[634081.247601],1.804,0.000000,-0.250354,[6746.76233637],[0.000479653871844],[2562.58740092],11,15.3846
004144,2014-08-01,[450508.50271],0.3807,0.002461,2.49376e+06,[5928.60860642],[0.000571711662852],[1782.63901761],8,13.0636
004786,2014-04-01,[186707.578349],0.1479,0.004533,0.0039323,[4342.13340869],[0.000889689671267],[97.05956433],4,19.6113
004909,2015-08-01,[192839.370969],-1.328,0.000000,2.89336e+06,[3506.66911123],[0.000948971159359],[-740.80476634],8,5.12949
005234,2016-01-01,[255738.762474],-0.8767,0.005500,0.107691,[4825.66298452],[0.000691036546652],[597.50449654],1,9.90154
005305,2016-08-01,[235802.172628],-0.8077,0.007664,0.138976,[4288.02036763],[0.000694391137937],[322.05657742],8,7.81103
